# Load necessary packages

In [2]:
# Load packages
from nilearn import datasets
from nilearn import input_data
import numpy as np
import pandas as pd

band_type = "Band"

dir_name = "~/Documents/COBRE/Raw Data/" + band_type +"_pass/"

# Set up the atlasses and data

In [3]:
# Get relevant data

## MSDL atlas
msdl = datasets.fetch_atlas_msdl( verbose = 0 )
msdl_map = msdl.maps
msdl_coords = msdl.region_coords
msdl_n_regions = len(msdl_coords)
msdl_labels = msdl.labels
print( "MSDL is a functional atlas with {} regions." .format( msdl_n_regions ) )

## AAL
aal = datasets.fetch_atlas_aal( verbose = 0 )
aal_map = aal.maps
aal_labels = aal.labels
aal_n_regions = len( aal.labels )
print( "AAL is an atomical atlas with {} regions." .format( aal_n_regions ) )

## COBRE data
n_subjects = None
cobre_data = datasets.fetch_cobre( n_subjects = n_subjects, verbose = 0 )
print( "There are {} subjects in the COBRE dataset." .format( len( cobre_data.func ) ) )

MSDL is a functional atlas with 39 regions.
AAL is an atomical atlas with 116 regions.
There are 146 subjects in the COBRE dataset.


/Users/matthewryan/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/npyio.py:2372: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  output = genfromtxt(fname, **kwargs)
/Users/matthewryan/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/npyio.py:2372: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  output = genfromtxt(fname, **kwargs)


In [11]:
cobre_data.func.sort()
cobre_data.confounds.sort()

# Create MSDL atlas

In [14]:
# Create and save MSDL atlas
msdl_atlas = input_data.NiftiMapsMasker( msdl.maps, resampling_target = "data", t_r = 2, detrend = True,
                                         standardize = True, memory = "nilearn_cache", 
                                         memory_level = 1 ).fit()

for i, (func_file, conf_file) in enumerate( zip( cobre_data.func, cobre_data.confounds ) ):
    file_name = dir_name + "MSDL/msdl_subject_" + str( i ) + ".csv"
    temp = msdl_atlas.transform( func_file, confounds = conf_file )
    temp_df = pd.DataFrame( temp, columns = msdl_labels )
    temp_df.to_csv( file_name, header = True, sep = "," )
    del file_name, temp, temp_df

# Create AAL Atlas

In [15]:
# Create and save AAL atlas
aal_atlas = input_data.NiftiLabelsMasker( aal.maps, resampling_target = "data", t_r = 2, detrend = True,
                                         standardize = True, memory = "nilearn_cache", 
                                         memory_level = 1 ).fit()

for i, (func_file, conf_file) in enumerate( zip( cobre_data.func, cobre_data.confounds ) ):
    file_name = dir_name + "AAL/aal_subject_" + str( i ) + ".csv"
    temp = aal_atlas.transform( func_file, confounds = conf_file )
    temp_df = pd.DataFrame( temp, columns = aal_labels )
    temp_df.to_csv( file_name, header = True, sep = "," )
    del file_name, temp, temp_df

# Create Power Atlas

In [16]:
power = datasets.fetch_coords_power_2011()
power_coords = np.delete(power.rois, 74, 0) # Drop 75 in conjunction with paper
power_atlas = input_data.NiftiSpheresMasker(seeds = power_coords[["x", "y", "z"]],
                                            radius = 10,
                                            t_r = 2,
                                            detrend = True,
                                            low_pass=0.1,
                                            high_pass=0.01,
                                            allow_overlap = True).fit()

In [17]:
for i, (func_file, conf_file) in enumerate( zip( cobre_data.func, cobre_data.confounds ) ):
    file_name = dir_name + "Power/power_subject_" + str( i ) + ".csv"
    temp = power_atlas.transform( func_file, confounds = conf_file )
    temp_df = pd.DataFrame( temp, columns = power_coords[["roi"]] )
    temp_df.to_csv( file_name, header = True, sep = "," )
    del file_name, temp, temp_df